In [ ]:
pip install langchain openai faiss-cpu unstructured pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.4/181.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 9.8 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223

In [ ]:
!pip install pymupdf sentence-transformers faiss-cpu llama-cpp-python

In [ ]:
pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00


In [ ]:
pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.5 MB/s eta 0:00:00


# Using GPT Model

# Load the PDF

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/A_Recommender_System_for_Healthy_Food_Choices_Building_a_Hybrid.pdf")
documents = loader.load()

# Split the Text into Chunks

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=100)
docs = splitter.split_documents(documents)


# Embed and Store in FAISS

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embedding = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embedding)
retriever = db.as_retriever()

<ipython-input-8-68621ca36582>:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


# RAG Chain

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0.3)
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)


<ipython-input-9-e020b8d461aa>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0.3)


In [ ]:
questions = [
    "What is a hybrid recommender system and how is it used?",
    "How Does the system deal with user calorie intake?",
    "What evaluation metrics were used in the study?"
]

for q in questions:
    print(f"Q: {q}")
    print("A:", rag_chain.invoke({"query": q}))
    print("-" * 80)


Q: What is a hybrid recommender system and how is it used?
A: {'query': 'What is a hybrid recommender system and how is it used?', 'result': 'A hybrid recommender system is an approach that combines multiple recommendation techniques to optimize the process of making suggestions to users. These systems aim to leverage the strengths and mitigate the weaknesses of individual recommendation methods, such as content-based and collaborative filtering, to improve the accuracy and relevance of recommendations. By integrating features of both (or multiple) types of recommender systems, hybrid models can provide more personalized and accurate suggestions.\n\nIn the context of the research on "A Recommender System for Healthy Food Choices," a hybrid recommender system is used to recommend healthier food options to individuals. This system combines content-based filtering, which focuses on the properties of items (in this case, recipes) and user preferences, with collaborative filtering, which re

# Using llama Model

##**Extract and Chunk PDF**

In [ ]:
import fitz  # PyMuPDF
import re

# Load PDF file and extract text
pdf_path = "/content/A_Recommender_System_for_Healthy_Food_Choices_Building_a_Hybrid (1).pdf"
doc = fitz.open(pdf_path)

full_text = ""
for page in doc:
    full_text += page.get_text()

# Split the text into 300-word chunks
def chunk_text(text, chunk_size=300):
    words = re.split(r'\s+', text)
    chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

chunks = chunk_text(full_text)
print(f"Number of chunks: {len(chunks)}")
print("Example chunk:\n", chunks[0])

Number of chunks: 23
Example chunk:
  A Recommender System for Healthy Food Choices: Building a Hybrid Model for Recipe Recommendations using Big Data Sets Pallavi Chavan, Brian Thoms, Jason Isaacs CSU Channel Islands pallaviamod.chavan289@myci.csuci.edu, brian.thoms@csuci.edu, jason.isaacs@csuci.edu Abstract Advances in Big Data analytics and machine learning have offered intangible benefits across many areas of one’s health. One such area is a move towards healthier lifestyle choices such as one’s diet. Recommender systems apply techniques that can filter information and narrow that information down based on user preferences or user needs and help users choose what information is relevant. Commonly adopted across e-commerce sites, social networking and entertainment industries, recommender systems can also support nutrition-based health management, offering individuals more food options, not only based on one’s preferred tastes but also on one’s dietary needs and restrictions. This r

##Generate Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(chunks, show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

##Store Embeddings in FAISS

In [ ]:
import faiss
import numpy as np

embeddings_np = np.array(embeddings).astype("float32")

# Create FAISS index
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)

##Load LLaMA Model and Define RAG Function

In [ ]:
# Download the model
from huggingface_hub import hf_hub_download
model_name = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
model_file = "tinyllama-1.1b-chat-v1.0.Q8_0.gguf"
model_path = hf_hub_download(repo_id=model_name, filename=model_file, local_dir="/content")
print(f"Model downloaded to: {model_path}")

tinyllama-1.1b-chat-v1.0.Q8_0.gguf:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Model downloaded to: /content/tinyllama-1.1b-chat-v1.0.Q8_0.gguf


In [ ]:
# Load the LLaMA model
from llama_cpp import Llama
llm = Llama(model_path=model_path, n_ctx=2048, verbose=False)
print("Model loaded successfully")

# Define retrieval + generation function (unchanged)
def rag_ask(question, k=3):
    question_embedding = model.encode([question]).astype("float32")
    # Search for top-k similar chunks
    distances, indices = index.search(question_embedding, k)
    context = "\n\n".join([chunks[i] for i in indices[0]])

    # Build the prompt for the LLM
    prompt = f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    output = llm(prompt, max_tokens=200, stop=["Question:"])
    print(" Answer:\n", output["choices"][0]["text"].strip())

Model loaded successfully


In [ ]:
rag_ask("What is a hybrid recommender system and how is it used?")
rag_ask("How does the system deal with user calorie intake?")
rag_ask("What evaluation metrics were used in the study?")

 Answer:
 In this research, a hybrid recommender system is used. A hybrid recommender system combines content-based and collaborative filtering techniques. The content-based model generates predictions based on the user’s diet and ingredient choices. The collaborative filtering model is used to filter predictions based on user preferences and diet restrictions. A gradual process is used to adjust the weights of the content-based model and collaborative filtering model. The results show that even if the performance of the hybrid model has increased slightly compared to collaborative filtering, it's more efficient in terms of recommendation because it takes into account user preferences and diet restrictions.
 Answer:
 The system does not deal with user calorie intake as it considers it only in the context of the user's preferences and dietary habits. It does not have access to the user's actual calorie intake and does not use calorie intake as a factor in recommendation.
 Answer:
 The s